## Problema da Produção de Móveis

**Problema da Produção de Móveis -** Um carpinteiro produz dois tipo de móveis, A e B, e dispõe de 90, 80 e 50 metros de compensado, pinho e cedro, respectivamente. Cada móvel do tipo A é vendido por 100,00 Reais e requer 2, 1 e 3 metros de compensado, pinho e cedro, respectivamente. Cada móvel do tipo B é vendido por 120,00 Reais e requer 1, 4 e 2 metros de compensado, pinho e cedro, respectivamente. 
Elabore um modelo matemático visando ajudar o carpinteiro a obter o maior lucro bruto possível.

x1: quantidade de móveis do tipo A a ser produzida.

x2: quantidade de móveis do tipo B a ser produzida.

max 100x1 + 120x2

sujeito a:

2x1 + x2 <= 90

x1 + 4x2 <= 80

3x1 + 2x2 <= 50

x1, x2 >= 0

x1, x2 ∈ Z

In [1]:
from docplex.mp.model import Model
import cplex

m = Model(name='Produção de Móveis')
x_1 = m.integer_var(name='x_1')
x_2 = m.integer_var(name='x_2')

m.add_constraint(2*x_1 + x_2 <= 90)
m.add_constraint(x_1 + 4*x_2 <= 80)
m.add_constraint(3*x_1 + 2*x_2 <= 50)

m.add_constraint(x_1 >= 0)
m.add_constraint(x_2 >= 0)

m.maximize(100*x_1 + 120*x_2)
m.solve()
print(m.solution)

solution for: Produção de Móveis
objective: 2680
x_1=4
x_2=19



### Usando o método de força bruta

1) testar, para cada combinação de m colunas de A, se elas formam uma base; 

2) calcular, para cada base, a solução básica associada a ela; 

3) calcular, para todas as SBV obtidas, o valor da função objetivo.

In [2]:
from math import factorial
from __future__ import print_function
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

**Forma Padrão:**

x1: quantidade de móveis do tipo A a ser produzida.

x2: quantidade de móveis do tipo B a ser produzida.

max 100x1 + 120x2

sujeito a:

2x1 + x2 + x3 = 90

x1 + 4x2 + x4 = 80

3x1 + 2x2 + x5 = 50

x1, x2, x3, x4, x5 >= 0

x1, x2, x3, x4, x5 ∈ Z

In [3]:
# Informar aqui os valores da matriz A
A = np.array([[2, 1, 1, 0, 0],
              [1, 4, 0, 1, 0],
              [3, 2, 0, 0, 1]])

In [4]:
# Informar aqui os valores da matriz b
b = np.array([[90], 
              [80], 
              [50]])

In [5]:
# Informar aqui os valores do vetor c
c = np.array([100, 120, 0, 0, 0])

In [6]:
m = len(A)
print('m:', m)

n = len(A[0])
print('n:', n)

combinacoes = factorial(n)/(factorial(m)*(factorial(n-m)))

print('O número máximo de soluções básicas:', combinacoes)

m: 3
n: 5
O número máximo de soluções básicas: 10


In [7]:
def verifica_solucao_viavel(x):
    '''
        Função que verifica se a solução é viável
        
        >>> verifica_solucao_viavel([0, 0, [125.], 0, [125.], [475.], 0])
            Esta é uma solução básica viável
            True
        >> verifica_solucao_viavel([0, 0, 0, [300.], [-950.], 0, [500.]])
            Esta NÃO é uma solução básica viável
            False
        >> verifica_solucao_viavel([1, 0, 1, 0, 1])
            Esta é uma solução básica viável
            True
    '''
    
    xB_bool = []

    for w in range(0, len(x)):
        if(x[w] >= 0):
            xB_bool.append(True)           
        else:
            xB_bool.append(False)

        try:
            if(x[w][0] >= 0):
                xB_bool.append(True)
            else:
                xB_bool.append(False)
        except:
            pass
            
    if(all(xB_bool)):
        print('Esta é uma solução básica viável')
        return True
    else:
        print('Esta NÃO é uma solução básica viável')
        return False

In [8]:
solucoes_viaveis = np.array([[0,0,0], 0,0,0])
todas_solucoes = np.array([[0,0,0], 0,0,0])
cont_x = 0

for w in range(0, n):
    for y in range(w+1, n):
        for z in range(y+1, n):
            
            # Id da solução
            cont_x = cont_x + 1;
            print('\nx', cont_x)
            
            # Conjunto de índices IB
            IB = [w,y,z]
            print('IB =', IB[0]+1, IB[1]+1, IB[2]+1)
            
            # B
            B = np.array([A[0][w], A[0][y], A[0][z]])
            for v in range (1, m):
                B = np.row_stack(tup=(B, [A[v][w], A[v][y], A[v][z]]))
            print('B =', B, '\n')
            
            # Verificando se B é uma base
            v, V =  np.linalg.eig(B)

            if(v.all() == 0):
                print('A matriz B não é uma base')
                print('A linha LD é:')
                print (B[v == 0,:])
                print('\n---------------------------------------------------------------------------\n')
                
                # Colocar na tabela de soluções básicas, com informações vazias
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                continue

            # B-1
            try:
                B1 = np.linalg.inv(B)
                print('B-1 =', B1, '\n')
            except:
                print("B-1 não existe")
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                print('\n---------------------------------------------------------------------------\n')
                continue

            # xB
            xB = B1.dot(b)
            print('xB =', xB, '\n')
            
            # x
            x = []
            for v in range (0, n):              
                if(v == IB[0]):
                    x.append(round(xB[0]))
                elif(v == IB[1]):
                    x.append(round(xB[1]))
                elif(v == IB[2]):
                    x.append(round(xB[2]))
                else:
                    x.append(0)
            print('x =', x, '\n')
                
            # cTx
            cTx = np.transpose(c).dot(x)
            print('cTx =', cTx, '\n')
                
            # Colocar na tabela de soluções básicas
            todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            # Se a solução for viável, colocar na tabela de soluções viáveis
            if(verifica_solucao_viavel(x) == True):
                solucoes_viaveis = np.row_stack((solucoes_viaveis, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            print('\n---------------------------------------------------------------------------\n')



x 1
IB = 1 2 3
B = [[2 1 1]
 [1 4 0]
 [3 2 0]] 

B-1 = [[ 0.  -0.2  0.4]
 [ 0.   0.3 -0.1]
 [ 1.   0.1 -0.7]] 

xB = [[ 4.]
 [19.]
 [63.]] 

x = [4.0, 19.0, 63.0, 0, 0] 

cTx = 2680.0 

Esta é uma solução básica viável

---------------------------------------------------------------------------


x 2
IB = 1 2 4
B = [[2 1 0]
 [1 4 1]
 [3 2 0]] 

B-1 = [[ 2.  0. -1.]
 [-3.  0.  2.]
 [10.  1. -7.]] 

xB = [[ 130.]
 [-170.]
 [ 630.]] 

x = [130.0, -170.0, 0, 630.0, 0] 

cTx = -7400.0 

Esta NÃO é uma solução básica viável

---------------------------------------------------------------------------


x 3
IB = 1 2 5
B = [[2 1 0]
 [1 4 0]
 [3 2 1]] 

B-1 = [[ 0.57142857 -0.14285714  0.        ]
 [-0.14285714  0.28571429  0.        ]
 [-1.42857143 -0.14285714  1.        ]] 

xB = [[ 40.]
 [ 10.]
 [-90.]] 

x = [40.0, 10.0, 0, 0, -90.0] 

cTx = 5200.0 

Esta NÃO é uma solução básica viável

---------------------------------------------------------------------------


x 4
IB = 1 3 4
B = [[2 1 0

In [9]:
print("Todas as soluções:")

todas_solucoes = pd.DataFrame(todas_solucoes[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base']).sort_values(by=['Id da solução'])
todas_solucoes

Todas as soluções:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,2680,"[4.0, 19.0, 63.0, 0, 0]"
1,"[1, 2, 4]",2,-7400,"[130.0, -170.0, 0, 630.0, 0]"
2,"[1, 2, 5]",3,5200,"[40.0, 10.0, 0, 0, -90.0]"
3,"[1, 3, 4]",4,1700,"[17.0, 0, 57.0, 63.0, 0]"
4,"[1, 3, 5]",5,8000,"[80.0, 0, -70.0, 0, -190.0]"
5,"[1, 4, 5]",6,4500,"[45.0, 0, 0, 35.0, -85.0]"
6,"[2, 3, 4]",7,3000,"[0, 25.0, 65.0, -20.0, 0]"
7,"[2, 3, 5]",8,2400,"[0, 20.0, 70.0, 0, 10.0]"
8,"[2, 4, 5]",9,10800,"[0, 90.0, 0, -280.0, -130.0]"
9,"[3, 4, 5]",10,0,"[0, 0, 90.0, 80.0, 50.0]"


In [10]:
qtd_solucoes_basicas = map(lambda z, sb=0: sb if z == [] else sb+1, todas_solucoes.loc[:, 'cTx']).count(1)

print(qtd_solucoes_basicas)

print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)

10

Quantidade de soluções básicas:  10


/home/myllah/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """Entry point for launching an IPython kernel.


In [11]:
print("Soluções viáveis:")

solucoes_viaveis = pd.DataFrame(solucoes_viaveis[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base']).sort_values(by=['Id da solução'])
solucoes_viaveis

Soluções viáveis:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,2680,"[4.0, 19.0, 63.0, 0, 0]"
1,"[1, 3, 4]",4,1700,"[17.0, 0, 57.0, 63.0, 0]"
2,"[2, 3, 5]",8,2400,"[0, 20.0, 70.0, 0, 10.0]"
3,"[3, 4, 5]",10,0,"[0, 0, 90.0, 80.0, 50.0]"


In [12]:
qtd_solucoes_viaveis = len(solucoes_viaveis)
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)


Quantidade de soluções viáveis:  4


In [13]:
## Digite 1 se você deseja maximizar ou 2 se você deseja minimizar
entrada = 1

if(entrada == 1):
    solucao_otima = solucoes_viaveis['cTx'].max(axis=0)
elif(entrada == 2):
    solucao_otima = solucoes_viaveis['cTx'].min(axis=0)
print('Solução Ótima:', solucao_otima)

Solução Ótima: 2680.0


In [14]:
print('\nSoluções que geram a solução ótima:\n')

geram_solucoes_otimas = solucoes_viaveis.loc[solucoes_viaveis['cTx'] == solucao_otima, :]
geram_solucoes_otimas


Soluções que geram a solução ótima:



,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,2680,"[4.0, 19.0, 63.0, 0, 0]"


In [15]:
print('\nO número máximo de soluções básicas:', combinacoes)
print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)
print('\nSolução Ótima:', solucao_otima)


O número máximo de soluções básicas: 10

Quantidade de soluções básicas:  10

Quantidade de soluções viáveis:  4

Solução Ótima: 2680.0
